In [15]:
import numpy as np

from halton_points import HaltonPoints
from function_Fm import *
from explicit_RK import *
from analytical_solution import *

from scipy.sparse import csc_matrix
from scipy.sparse.linalg import gmres

## Boundary

In [16]:
nf = (2**0+1)**2
r = HaltonPoints(2, nf).haltonPoints()
fxl = r.copy()
fxl[:, 0] = 0
fxr = r.copy()
fxr[:, 0] = 1
fyu = r.copy()
fyu[:, 1] = 0
fyd = r.copy()
fyd[:, 1] = 1
Mb = np.vstack((fxl, fxr, fyu, fyd))

## Interior

In [17]:
poly_b = np.array([[-1, -1, 1], [1/2, 3/2, -1], [3/2, 1/8, -3/8]])
npnts = (2**4+1)**2

uh = solve_matrix(Mb=Mb, npnts=npnts, poly_b=poly_b, rbf='TPS')
uh.exact_solution = "2"
uh.nu = 0.001
uh.c = 1/4
uh.pols = 'Hermite'

domain = create_domain(uh.Mi, radius=0.35, c_y=0.5, c_x=0.5)

#unit_square
#circle_centre

uh.Mi, new_boundary = domain.setup(domain='unit_square', bound_points=12)
uh.Mb = np.vstack((uh.Mb, new_boundary))
uh.ni = uh.Mi.shape[0]
uh.nb = uh.Mb.shape[0]
X0 = uh.X_0()

In [18]:
print("Interior points: {:,.0f} \nBoundary points: {:,.0f} \nTotal points: {:,.0f}".format(uh.Mi.shape[0], uh.Mb.shape[0], uh.Mi.shape[0] + uh.Mb.shape[0]))

Interior points: 289 
Boundary points: 16 
Total points: 305


In [19]:
#uh.epsilon = 5.956
cond = np.linalg.cond(uh.ACaps())
det = np.linalg.det(uh.ACaps())
print('Condition number: {:,.0f} \nDeterminant: {:,.9f}'.format(cond, det))

Condition number: 54 
Determinant: -177,348,695,872,234.562500000


# Solve Matrix RK-Newthon

In [20]:
Y = np.vstack((X0, X0))
dt = 0.1
uh.variables(X0, dt, Y)
#rr = uh.Rk(0.02)

In [21]:
for t in range(1,11):
    error = list()
    for _ in range(10):
        uh.Y = Y
        A, b = uh.inexact_Newthon(t/10)
        u, exitCodeu = gmres(A, b[:,0].reshape(-1,1))
        v, exitCodev = gmres(A, b[:,1].reshape(-1,1))
        Sp = np.hstack((u.reshape(-1,1), v.reshape(-1,1)))
        Y = Y - Sp

    Xk = uh.step(t/10)
    u_exact = exact_solution(uh.Mi, uh.nu).u(t/10)
    e = np.linalg.norm(u_exact - Xk)/np.linalg.norm(u_exact)
    print('tiempo: {} \t error: {:,.6f}'.format(t/10, e))
    error.append(e)
    uh.Xk = Xk
    uh.Y = np.vstack((Xk, Xk))

tiempo: 0.1 	 error: 0.037423
tiempo: 0.2 	 error: 0.062549


In [ ]:
error

In [ ]:
# for _ in range(50):
#     A, b = uh.inexact_Newthon(0.01)
#     Sp = np.linalg.inv(A).dot(b)
#     Y =  Y - Sp
#     uh.Y = Y
#     print(np.linalg.norm(Y, 'fro'))